In [1]:
import re
from datetime import datetime

In [2]:
cur_identifier = None
cur_alignment = None
cur_results = None
cur_state = ""
last_datetime_stored = None
last_start_time = None
all_results = []
with open("../preprocessed/bash_log.txt", "r") as handle:
    for line in handle:
        line = line.rstrip()
        try:
            last_datetime_stored = datetime.strptime(line[:26], '%Y-%m-%d %H:%M:%S.%f')
        except:
            pass
        if cur_state == "":
            match_identifier = re.search("^-+sample-(?P<n>\d+)-(?P<i>\d+)-+$", line)
            if match_identifier:
                cur_identifier = (int(match_identifier.group("n")), int(match_identifier.group("i")))
                cur_state = "parse_alignment"
                cur_alignment = None
                cur_results = {}
        elif cur_state == "parse_alignment":
            m = re.search("^(.+):  Predicting domains for \d+ fasta files$", line)
            if m:
                cur_results["start_hmmscan"] = datetime.strptime(m.group(1), '%Y-%m-%d %H:%M:%S.%f')
                continue
            m = re.search("^(.+):  Finished generating domtable files.$", line)
            if m:
                cur_results["end_hmmscan"] = datetime.strptime(m.group(1), '%Y-%m-%d %H:%M:%S.%f')
                if "start_hmmscan" in cur_results:
                    cur_results["hmmscan"] = (cur_results["end_hmmscan"] - cur_results["start_hmmscan"]).total_seconds()
                continue
            m = re.search("^launch_(.+) took (.+) seconds$", line)
            if m:
                cur_alignment = m.group(1)
                cur_results["alignment"] = float(m.group(2))
                continue
            m = re.search("^\s+Main function took (.+) s$", line)
            if m:
                cur_results["main_function"] = float(m.group(1))
                continue
            if line == "   - - Processing input files - -":
                if len(cur_results.keys()) > 0:
                    runtime = (last_datetime_stored - last_start_time).total_seconds()
                    all_results.append([
                        cur_identifier[0],
                        cur_identifier[1],
                        cur_alignment,
                        cur_results.get("hmmscan"),
                        cur_results.get("alignment"),
                        cur_results.get("main_function"),
                        runtime
                    ])
                    cur_alignment = None
                    cur_results = {}
                last_start_time = last_datetime_stored
                continue
            m = re.search("^-+sample-(\d+)-(\d+)-+$", line)
            if m:
                if len(cur_results.keys()) > 0:
                    runtime = (last_datetime_stored - last_start_time).total_seconds()
                    all_results.append([
                        cur_identifier[0],
                        cur_identifier[1],
                        cur_alignment,
                        cur_results.get("hmmscan"),
                        cur_results.get("alignment"),
                        cur_results.get("main_function"),
                        runtime
                    ])
                    cur_alignment = None
                    cur_results = {}
                cur_state = ""
                continue

In [6]:
all_results = sorted(all_results, key=lambda x: (x[0], x[1]))
from IPython.display import HTML, display
display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in all_results)
       )
))

100,1,hmmalign,None,20.583,None,26.29966
100,1,muscle,None,2.627,None,8.284079
100,1,mafft,None,97.206,None,5.079237
100,2,hmmalign,None,17.572,None,23.279073
100,2,muscle,None,2.128,None,8.032597
100,2,mafft,None,83.285,None,5.18886
100,3,hmmalign,None,18.374,None,23.840312
100,3,muscle,None,1.724,None,7.27133
100,3,mafft,None,72.968,None,5.186147
100,4,hmmalign,None,14.961,None,20.600839
100,4,muscle,None,1.728,None,7.222509
